# Mechanics Applications in FEniCS

This lecture is based on the [FEniCS tutorials](https://fenicsproject.org/pub/tutorial/html/._ftut1001.html#table_of_contents).

In [31]:
%matplotlib inline

from fenics import *
import numpy as np
import matplotlib.pylab as plt

| <img src="figures/heart.jpg" width=250> |
|:--:|
| *The heart is perfused by thousands of small blood vessels. Photo of a resin cast by Arno Massee* |

Oxygen and other nutrients are vital supplies for tissue function in general and are delivered via the cardiovascular system. The arteries supplying tissue branch into thousands of small arteries, arterioles and capillaries that are impossible to capture using modern imaging techniques. However, averaging over entire blocks of tissue allows us to approximate perfusion as a spongy material and to thus exploit porous continuum models.

## Tissue perfusion: Darcy's law

The previous lecture introduced porous flow in the form of Darcy's law. In its reduced form Darcy's law reads

$$\nabla \cdot \left( -\boldsymbol{K} \; \nabla p(\boldsymbol{x}) \right) = f(\boldsymbol{x}) \qquad \text{on } \Omega$$

augmented by some Dirichlet boundary conditions

$$p(\boldsymbol{x}) = p_D(\boldsymbol{x}) \qquad \text{on } \partial \Omega$$

In this equation we solve for the unknown pressure $p$ only. Velocity a calculated from the result as

$$u = -\boldsymbol{K} \; \nabla p(\boldsymbol{x}) \qquad \text{on } \Omega$$

Note that $u$ is the velocity of fluid _within a pore_. To obtain velocity on the tissue scale, $u$ has to be scaled by the tissue's porosity

$$\bar{u} = \phi u$$

Darcy's law itself is a stationary problem (no time dependence). However, in mechanics applications, and for biological systems in particular, we are most often interested in solving time-dependent problems. Darcy's law becomes time-dependent when either $p_D$ or $f(x)$ (or both!) are functions of time. In this case, each timestep is solved as a stationary sub-problem, while $p_D$ and/or $f(x)$ are updated at the beginning of each timestep. We can thus easily expand our code for solving Darcy's law.

### FEniCS implementation

We first set up the problem parameters and variational forms

In [32]:
# Create mesh and define function space
n = 10
mesh = UnitSquareMesh(n, n)
W = FunctionSpace(mesh, 'P', 1) # pressure space
V = VectorFunctionSpace(mesh, 'P', 2)

# Define permeability and porosity
kmin = 0.01
K = Expression('max(exp(-pow((x[1]-0.5-0.1*sin(10*x[0]))/0.1, 2)), 0.01)', degree=1)
phi = Constant(0.2)

# Define boundary condition
pD = Expression('(1-x[0])*t', t=0.0, degree=1)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(W, pD, boundary)

# Define variational problem
p = TestFunction(W)
q = TrialFunction(W)
f = Constant(0.0)
n = FacetNormal(mesh)
a = K * inner(grad(p), grad(q))*dx
L = f*q*dx

We will use the XDMF file format to store results. An example of how an XDMF file can store a function is shown below:

In [33]:
p = Function(W)
file = XDMFFile("perfusion/test.xdmf")
# do some stuff
file.write_checkpoint(p, 'pressure', t, append=False)
# do some more stuff
file.close()

The function write_checkpoint() takes 3 arguments, plus one optional argument. The arguments are the FEniCS Function to be stored, a label for the function, and time. The optional argument can be used to set the file write mode to "append" (default append=False). Files are closed at the very end of the code.

#### Exercise: Complete the code

Complete the code below such that the variational form above is solved until $t = 1.0$ using the given time step size. Store $p$ and $u$ in the corresponding XDMF file during each time step. XDMF files can be postprocessed / viewed in [Paraview](https://www.paraview.org/).

In [34]:
p_file = XDMFFile("perfusion/p.xdmf")
u_file = XDMFFile("perfusion/u.xdmf")

# Compute solution
dt = 0.1
t = 0.0
append = False
p = Function(W)
u = Function(V)

# Insert code here
    
p_file.close()
u_file.close()

#### Exercise: Visualisation in Paraview

- Opening files
- Visualising scalar quantities
- Visualising vector quantities
- Using filters

#### Exercise: Error analysis

## Tracking tracers: Advection-diffusion-reaction equations

While fluid pressure and velocity within biological tissues are interesting quantities in their own respect, it is often particularly interesting to know how tracer molecules distribute through the tissue, driven by fluid dynamics. This is achieved via a system of advection-diffusion-reaction equations.

## Elastic deformation: Linear elasticity

Linear elasticity deals with the equations governing small elastic deformations following a linear relationship between stress and strain. Nonlinear stress-strain relationships are goverened by the hyperelastic equations, which are introduced in the Cardiac Tissue Mechanics and Fluid Dynamics stream of the Summer School.

The momentum balance equation for small elastic deformations of a body $\Omega$ reads

$$-\nabla \cdot \sigma = f \qquad \text{in } \Omega$$

with stress tensor $\sigma$, elasticity tensor $\varepsilon$ and body force per unit volume $f(\boldsymbol{x})$. 

$$\sigma = \lambda \; tr(\varepsilon) I + 2 \mu \varepsilon$$
$$\varepsilon = \frac{1}{2} (\nabla u + (\nabla u)^T)$$

The stress tensor depends on two parameters that are specific for every material, the Lamè constants $\lambda$ and $\mu$. The trace operator $tr(A)$ represents the sum of the elements on the main diagonal of a tensor $A$, $I$ is the identity tensor, and $u(\boldsymbol{x})$ is the displacement vector field. Inserting the definition of $\varepsilon$ into $\sigma$ gives

$$\sigma = \lambda (\nabla \cdot u) I + \mu (\nabla u + (\nabla u)^T)$$

Note that while we have written the momentum balance equation in terms of the stress tensor $\sigma$, our primary unknown is the displacement vector field $u$.

### Variational form

Deriving the variational form requires multiplication of the governing equation by a test function $v \in \hat{V}$, integrating over the domain $\Omega$.

$$- \int_{\Omega} (\nabla \cdot \sigma) \cdot v dx = \int_{\Omega} f \cdot v dx$$

Note that this time, as our unknown $u$ is vector-valued, our test function $v$ is also vector-valued. Note also from the definition of $\sigma$ that the term $\nabla \cdot \sigma$ contains second-order derivatives of $u$, so we integrate this term by parts

$$- \int_{\Omega} (\nabla \cdot \sigma) \cdot v dx = \int_{\Omega} \sigma : \nabla v dx - \int_{\partial \Omega} (\sigma \cdot n) \cdot v ds$$

where the colon operator denotes the *inner product* between two tensors:

$$A : B = \sum_i \sum_j A_{ij} B_{ij} = A_{11} B_{11} + A_{12} B_{12} + \ldots$$

The term $\sigma \cdot n$ denotes traction or stress at the boundary in the outward normal direction of the boundary and is often prescribed as a boundary condition. In fluid-structure interaction problems such as the deformation of an arterial wall due to the oscillating blood pressure, the boundary traction is directly proportional to the fluid pressure $\sigma \cdot n = -p n$. Here, we assume that traction is prescribed on some part $\partial \Omega_T$ of the boundary as $\sigma \cdot n = T$. On the remaining boundary we assume that displacement is prescribed using a Dirichlet boundary condition. The variational form thus reads

$$\int_{\Omega} \sigma : \nabla v dx = \int_{\Omega} f \cdot v dx - \int_{\partial \Omega_T} T \cdot v ds$$

or

$$a(u,v) = L(v) \qquad \forall v \in \hat{V}$$

where

$$a(u,v) = \int_{\Omega} \sigma : \nabla v dx$$
$$\sigma = \lambda (\nabla \cdot u) I + \mu (\nabla u + (\nabla u)^T)$$
$$L(v) = \int_{\Omega} f \cdot v dx - \int_{\partial \Omega_T} T \cdot v ds$$

The inner product of a symmetric tensor $A$ with an antisymmetric tensor $B$ vanishes. By expressing $\nabla v$ as a sum of its symmetric and antisymmetric parts it is thus possible to simplify the term $\sigma : \nabla v$ since $\sigma$ is always a symmetric tensor to obtain

$$a(u,v) = \int_{\Omega} \sigma : \varepsilon(v) dx$$

where

$$\varepsilon(v) = \frac{1}{2} (\nabla v + (\nabla v)^T)$$

is the symmetric part of $\nabla v$.

### FEniCS implementation

As an example, we will consider the elastic deformation of the wall of an artery, represented by an axisymmetric cylindrical tube. 